# MAVERICK - Inferencia Demo

Új képek predikciója OCR + NER támogatással.

In [ ]:
# Setup
from google.colab import drive
drive.mount('/content/drive')

!git clone https://github.com/Koppi02/AI-Agent.git
%cd AI-Agent
!pip install -q -r requirements.txt

In [ ]:
import sys
sys.path.insert(0, '/content/AI-Agent')

import tensorflow as tf
from src.data.loader import DataLoader
from src.models.predictor import MaverickPredictor
from src.utils.io_utils import load_config

In [ ]:
# Konfiguráció
config = load_config('config/config.yaml')
config['data']['drive_path'] = '/content/drive/MyDrive/AI_PROJECTS/'

## Predictor Inicializálása

In [ ]:
# Adatok betöltése
data_loader = DataLoader(config)
abs_df = data_loader.load_abs_torveny()
labels_df = data_loader.load_labels(use_processed=True)

# Modell betöltése
model_path = config['checkpoints']['model_save_path']
model = tf.keras.models.load_model(model_path)

# Predictor létrehozása
predictor = MaverickPredictor(
    config=config,
    model=model,
    index_to_segment=data_loader.index_to_segment,
    index_to_brand=data_loader.index_to_brand,
    index_to_basebrand=data_loader.index_to_basebrand,
    index_to_advertiser=data_loader.index_to_advertiser,
    abs_torveny_df=abs_df,
    abs_torveny_path=config['data']['drive_path'] + config['data']['abs_torveny_csv']
)

print("? Predictor készen áll!")

## Predikció Tesztelése

In [ ]:
# Kép útvonal megadása
test_image = '/content/drive/MyDrive/AI_PROJECTS/test_images/02183256.jpg'

# Predikció
result = predictor.predict_with_new_category_support(
    image_path=test_image,
    enable_ocr=True
)

In [ ]:
# Eredmények megjelenítése
print("\n" + "="*70)
print("?? PREDICTION RESULTS")
print("="*70)
print(f"Segment:    {result['segment']} ({result['segment_confidence']:.2%})")
print(f"Brand:      {result['brand']} ({result['brand_confidence']:.2%})")
print(f"BaseBrand:  {result['basebrand']} ({result['basebrand_confidence']:.2%})")
print(f"Advertiser: {result['advertiser']} ({result['advertiser_confidence']:.2%})")

if result['is_new_advertiser']:
    print("\n?? ÚJ ADVERTISER ÉSZLELVE!")
    print(f"Javasolt név: {result['advertiser']}")
    print(f"OCR bizonyíték: {result['ocr_text'][:100]}...")

print("="*70)

## Új Advertiser Jóváhagyása

In [ ]:
# Ha van új advertiser, jóváhagyhatod
if result['requires_approval']:
    approve = input("Jóváhagyod az új advertisert? (igen/nem): ")
    
    if approve.lower() == 'igen':
        success = predictor.approve_new_advertiser(
            result['proposal_id'],
            user_approved=True
        )
        
        if success:
            print("? Új advertiser elmentve az ABS_torveny.csv-be!")
        else:
            print("? Hiba történt a mentés során.")

## Batch Predikció

In [ ]:
# Több kép egyszerre
import os
import pandas as pd

test_folder = '/content/drive/MyDrive/AI_PROJECTS/test_images/'
test_images = [os.path.join(test_folder, f) for f in os.listdir(test_folder) if f.endswith('.jpg')]

results_list = []

for img_path in test_images[:5]:  # Elsõ 5 kép
    result = predictor.predict_with_new_category_support(img_path, enable_ocr=True)
    results_list.append({
        'image': os.path.basename(img_path),
        'segment': result['segment'],
        'brand': result['brand'],
        'advertiser': result['advertiser'],
        'is_new': result['is_new_advertiser']
    })

# DataFrame formázás
results_df = pd.DataFrame(results_list)
display(results_df)